## Prediction of sales in Europe Video Games market.
As Cyberpunk 2077 thought us the stock value of Video Game market is strictly dependent on number of sales.  
This experiment helps yo to predict number of sales to lower the risk of stock investements.  


### Configure azureml sdk. 
Config should be autoimported from Notebook context.  
If you encounter problems try to download json config from your Directories menu and place it next to Notebook in config.json file.

In [1]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

### Import and visualize dataset
Download sample dataset for video games sales.  
Sample could be found at https://www.kaggle.com/pranavkrishna/vgsales.  
Name dataset as 'games-sales'

In [3]:
aml_dataset = ws.datasets['games-sales']

# Use Pandas DataFrame just to sneak peak some data and schema
full_df = aml_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
full_df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Describe dataset.

In [4]:
# Use Pandas DataFrame just to investigate the dataset's schema and info
full_df.describe()

,Rank,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,41.490000,29.020000,10.220000,10.570000,82.740000


### Filter dataset for improved prediction.

It is essential for solving regression problems to remove excessive data.  
It eases learning process and improve accuracy.

Dropping Genre type as we are predicting total sells.

In [5]:
# Dropping Genre type as we are predicting total sells.
  
aml_dataset = aml_dataset.drop_columns([
    'Genre','genre'])

full_df = aml_dataset.to_pandas_dataframe()
full_df.describe()

,Rank,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,41.490000,29.020000,10.220000,10.570000,82.740000


### Split data into learn and test datasets

In [6]:
# Split using Azure Tabular Datasets (Better for Remote Compute)

train_dataset, test_dataset = aml_dataset.random_split(0.9, seed=1)

# Use Pandas DF only to check the data
train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())
# print(test_dataset_df.describe())

               Rank      NA_Sales      EU_Sales      JP_Sales   Other_Sales  \
count  15031.000000  15031.000000  15031.000000  15031.000000  15031.000000   
mean    8308.894219      0.264796      0.146325      0.078474      0.048367   
std     4786.003478      0.836679      0.515704      0.313870      0.194533   
min        1.000000      0.000000      0.000000      0.000000      0.000000   
25%     4158.000000      0.000000      0.000000      0.000000      0.000000   
50%     8324.000000      0.080000      0.020000      0.000000      0.010000   
75%    12445.500000      0.240000      0.110000      0.040000      0.030000   
max    16600.000000     41.490000     29.020000     10.220000     10.570000   

       Global_Sales  
count  15031.000000  
mean       0.538240  
std        1.591495  
min        0.010000  
25%        0.060000  
50%        0.170000  
75%        0.470000  
max       82.740000  


#### (Optional) list your workspace in order to ease further configuration.

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[{
   "id": "/subscriptions/5c191f39-6dbb-49fe-aabe-f85e875adb65/resourceGroups/covid-mlg/providers/Microsoft.MachineLearningServices/workspaces/covid-wsp/computes/covid-inst",
   "name": "covid-inst",
   "location": "westeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "westeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS11_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://covid-inst.westeurope.instances.azureml.ms"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://covid-inst.westeurope.instances.azureml.ms/lab"
         },
         {
           "displayName": "RStudio",
           "endpointUri": "https://covid-inst-8787.westeurope.instances.azureml.ms"
         }
       ],
       "con

### Connect to remote AML compute instance.
Ensure your enginners created compute cluster formerly.  
If not set up CREATE_CLUSTER_IF_NOT_FOUND if you want to create compute claster by script.  
If so, then set up proper NEW_CLUSTER_VM name.

In [8]:
# Choose a name for your cluster.
amlcompute_cluster_name = "covid-clst"

CREATE_CLUSTER_IF_NOT_FOUND = False
NEW_CLUSTER_VM = 'vm'
found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if (not found) & CREATE_CLUSTER_IF_NOT_FOUND:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = NEW_CLUSTER_VM,
                                                                 max_nodes = 20)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

Found existing training cluster.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
# For additional details of current AmlCompute status:
aml_remote_compute.get_status().serialize()

{'currentNodeCount': 0,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 0,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Resizing',
 'allocationStateTransitionTime': '2021-02-01T11:11:56.796000+00:00',
 'errors': None,
 'creationTime': '2021-01-31T18:27:55.335234+00:00',
 'modifiedTime': '2021-01-31T18:28:10.732323+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 0,
  'maxNodeCount': 2,
  'nodeIdleTimeBeforeScaleDown': 'PT120S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_DS11_V2'}

#### (Optional) List and select primary metric to drive the AutoML classification problem.

In [10]:
from azureml.train import automl

# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('regression')

# We'll use 'accuracy' as primary metric (Closer to 1.00 is better)

['normalized_root_mean_squared_error',
 'normalized_mean_absolute_error',
 'r2_score',
 'spearman_correlation']

### Define AutoML Experiment settings (With AML Remote Compute)

LightGBM has the best score for given data, so limit evaluated models to the best one. (`whitelist_models`)  
Use `normalized_root_mean_squared_error` as primary_metric.

In [11]:
import logging
import os

from azureml.train.automl import AutoMLConfig

project_folder = './vgs-resources'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=45,                            
                             training_data=train_dataset,
                             label_column_name="EU_Sales",
                             n_cross_validations=5,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder,
                             whitelist_models=['LightGBM']
                             )

#### Run Experiment

In [12]:

from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "predict-vg-sales-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

predict-vg-sales-02-01-2021-11
Running on remote.
No run_configuration provided, running on covid-clst with default configuration
Running on remote compute: covid-clst
Parent Run ID: AutoML_25789e53-94e9-4788-87b6-a243eee64de0



### (Optional) Attempt to handle widgets SyntaxError.
For some reason `SyntaxError: Unexpected token N in JSON at position 33259` occurs in RunDeetails class.  
If you are getting this error and pip install does not help you can continue to next steps.  

In [ ]:
pip install azureml-widgets

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

### Measure Parent Run Time needed for the whole AutoML process

In [ ]:
import time
import datetime as dt

run_details = run.get_details()

# Like: 2020-01-12T23:11:56.292703Z
end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))
print(run_details)

### Retrieve the 'Best Model' (LightGBM model)

In [ ]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

### Extract X values (feature columns)

In [ ]:
x_test_df = test_dataset_df
y_test_df = test_dataset_df

### Make the actual Predictions

In [ ]:

# Try the best model
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])
y_predictions.shape

#### Calculate the Accuracy with Test Dataset

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
accuracy_score(y_test_df, y_predictions)